In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
import os


In [ ]:
traindir='./age_utk_v2'

print('Training classes')
print(os.listdir(traindir))


In [ ]:
height = 200
width = 200
train_datagen =  ImageDataGenerator(rescale = 1/255.0,rotation_range=5,height_shift_range=0.5,shear_range=0.5,
                              zoom_range=0.2,validation_split=0.2,horizontal_flip=True)

train_data = train_datagen.flow_from_directory(directory = traindir,target_size=(height,width),
                                               class_mode = "categorical",batch_size=32,subset="training")

val_datagen = ImageDataGenerator(rescale = 1/255.0,rotation_range=15,height_shift_range=0.5,shear_range=0.5,
                              zoom_range=0.2,validation_split=0.2,horizontal_flip=True)

val_data = val_datagen.flow_from_directory(directory = traindir,target_size=(height,width),
                                               class_mode = "categorical",batch_size=32,subset="validation")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPool2D,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

ResNet Model

In [ ]:
from tensorflow.keras.applications import ResNet50
resnet = ResNet50(weights = "imagenet",include_top = False,input_shape=(height,width,3))

for layer in resnet.layers:
    layer.trainable = False

model = Sequential()
model.add(resnet)
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(4,activation = "softmax"))

model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=0.001),
              loss="categorical_crossentropy",
              metrics="accuracy")

In [ ]:
checkpoint=ModelCheckpoint("age_prediction4_v5.h5",
                           monitor="val_accuracy",
                           save_best_only=True, 
                           verbose=1)

#earlystop=EarlyStopping(monitor="val_accuracy",
                       #patience=5,
                       #verbose=1)

In [ ]:
batch_size=32

train_step=len(train_data)/batch_size
val_step=len(val_data)/batch_size

result=model.fit(train_data,
                 steps_per_epoch=train_step,
                 epochs=10,
                 validation_data=val_data,
                 validation_steps=val_step,
                 callbacks=[checkpoint],
                 verbose=1)

In [ ]:
import matplotlib.pyplot as plt

# Training results of network



plt.plot(result.history['accuracy'])
plt.plot(result.history['val_accuracy'])


# Set the parameters
plt.title('Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper right')

# Display the plots
plt.show()

plt.plot(result.history['loss'])
plt.plot(result.history['val_loss'])



# Set the parameters
plt.title('Deep learning model loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'], loc='upper right')

# Display the plots
plt.show()

Custom Network Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping

In [ ]:
from keras import regularizers
input_shape=(200,200,3)

model = Sequential()
model.add(Conv2D(16,kernel_size=(3,3),activation='relu',padding='same',kernel_regularizer=regularizers.l2(0.001),input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())


model.add(Conv2D(32,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.001),padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())

model.add(Conv2D(64,kernel_size=(3,3),activation='relu',kernel_regularizer=regularizers.l2(0.001),padding='same'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())


model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(5,activation = "softmax"))

model.summary()

In [ ]:
model.compile(optimizer=Adam(),
              loss="categorical_crossentropy",
              metrics="accuracy")

In [ ]:
#checkpoint=ModelCheckpoint("ageprediction_utk.h5",
                           monitor="val_accuracy",
                           save_best_only=True, 
                           verbose=1)

#earlystop=EarlyStopping(monitor="val_accuracy",
                       #patience=5,
                       #verbose=1)

In [ ]:
batch_size=32

train_step=len(train_data)//batch_size
val_step=len(val_data)//batch_size

result=model.fit(train_data,
                 steps_per_epoch=70,
                 epochs=50,
                 validation_data=val_data,
                 validation_steps=70,
                 #callbacks=[checkpoint],
                 verbose=1)

In [ ]:
#output of training

plt.plot(result.history['val_accuracy'])
plt.plot(result.history['accuracy'])

#set the parameters
plt.title('Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train','validation'], loc='upper right')

plt.show()

#loss function

plt.plot(result.history['val_loss'])
plt.plot(result.history['accuracy'])

plt.title('Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train','validation'],loc='upper right')